In [ ]:
!git clone https://github.com/GoGoDuck912/Self-Correction-Human-Parsing.git
%cd Self-Correction-Human-Parsing

In [ ]:
import os
# Create the directory if it doesn't exist
os.makedirs("/kaggle/working/human-parsing", exist_ok=True)

import os

# Create the directory if it doesn't exist
os.makedirs("/kaggle/working/cloth-mask", exist_ok=True)

import os

# Create the directory if it doesn't exist
os.makedirs("/kaggle/working/json_keypoints", exist_ok=True)

In [ ]:
# Install the modern library and other dependencies
!pip install inplace-abn
!pip install -r requirements.txt

# --- CRITICAL FIXES ---

# 1. Delete the repository's outdated 'modules' folder
print("Deleting outdated 'modules' folder...")
!rm -rf ./modules

# 2. Edit the script to import the modern library instead
print("Fixing import statement...")
!sed -i 's/from modules import InPlaceABNSync/from inplace_abn import InPlaceABNSync/' ./networks/AugmentCE2P.py

# 3. Fix the incompatible activation function name
print("Fixing activation function name...")
!sed -i "s/activation='none'/activation='identity'/g" ./networks/AugmentCE2P.py

print("\nAll fixes applied successfully!")

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib

In [ ]:
!pip install tfjs-graph-converter

In [ ]:
!git clone https://github.com/rwightman/posenet-pytorch  # fix bugs
%cd posenet-pytorch

In [ ]:
def parsing_map(image_dir, output_dir="/kaggle/working/human-parsing"):
    """
    Runs the human parsing script from the correct directory.
    """
    # This command assumes you've already run the setup cell that clones the repo
    # and installs dependencies.
    %cd /kaggle/working/Self-Correction-Human-Parsing
    
    !python3 simple_extractor.py \
        --dataset lip \
        --model-restore /kaggle/input/lip-modellll/exp-schp-201908261155-lip.pth \
        --input-dir {image_dir} \
        --output-dir {output_dir}

In [ ]:
parsing_map('/kaggle/input/data-of-user/data/image')

CLOTH MASK

In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
import os

def cloth_mask(cloth_path, output_dir="/kaggle/working/cloth-mask"):
    """
    Creates a segmentation mask for a single cloth image.
    """
    # (SAM model loading logic is the same...)
    sam_checkpoint = "/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth"
    model_type = "vit_b"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
    predictor = SamPredictor(sam)

    image = cv2.imread(cloth_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image)
    
    h, w = image.shape[:2]
    input_point = np.array([[w // 2, h // 2]])
    input_label = np.array([1])
    
    masks, scores, _ = predictor.predict(
        point_coords=input_point, point_labels=input_label, multimask_output=True)
        
    best_mask = (masks[np.argmax(scores)] * 255).astype(np.uint8)
    
    # <-- FIX: Create a dynamic output name.
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.basename(cloth_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_mask.png"
    output_path = os.path.join(output_dir, output_filename)
    
    cv2.imwrite(output_path, best_mask)
    print(f"Saved cloth mask to {output_path}")


In [ ]:
# Now, calling the function will work correctly
cloth_mask('/kaggle/input/data-of-user/data/cloth/00000_00.jpg')

In [ ]:
import os
import json
import torch
import numpy as np
import posenet
from posenet.models.model_factory import load_model

def keypoints(image_path, output_dir="/kaggle/working/json_keypoints"):
    """
    Generates pose keypoints for a single image and saves them to a JSON file.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    net = load_model(101).cuda()
    output_stride = net.output_stride
    
    input_image, _, _ = posenet.read_imgfile(
        image_path, scale_factor=1.0, output_stride=output_stride)

    with torch.no_grad():
        input_image = torch.Tensor(input_image).cuda()
        heatmaps, offsets, fwd, bwd = net(input_image)
        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps.squeeze(0), offsets.squeeze(0), fwd.squeeze(0), bwd.squeeze(0),
            output_stride=output_stride, max_pose_detections=1, min_pose_score=0.25)
    
    poses = keypoint_coords.astype(np.int32)
    if not len(poses) > 0:
        print(f"Warning: No pose detected for {image_path}")
        return None

    pose = poses[0]
    
    indices = [0, (5, 6), 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]
    openpose = []
    for ix in indices:
        if ix == (5, 6):
            openpose.append([int((pose[5][1] + pose[6][1]) / 2), int((pose[5][0] + pose[6][0]) / 2), 1])
        else:
            openpose.append([int(pose[ix][1]), int(pose[ix][0]), 1])

    coords = [float(item) for sublist in openpose for item in sublist]
    data = {"people": [{"pose_keypoints_2d": coords}]}
    
    base_name = os.path.basename(image_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_keypoints.json"
    output_path = os.path.join(output_dir, output_filename)
    
    with open(output_path, 'w') as f:
        json.dump(data, f)
    
    return output_path

In [ ]:
keypoints('/kaggle/input/data-of-user/data/image/00009_00.jpg')

In [ ]:
import cv2
import json
import numpy as np
import os # <-- Added the missing import

def open_pose(image_path, keypoints_path, output_dir="/kaggle/working/openpose_image"):
    """
    Draws a pose skeleton on a blank canvas using keypoints from a JSON file.

    Args:
        image_path (str): Path to the original image to get canvas dimensions.
        keypoints_path (str): Path to the JSON file containing the pose keypoints.
        output_dir (str): Directory where the output skeleton image will be saved.
    """
    # --- Load image using the function argument ---
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return
        
    canvas_height, canvas_width = img.shape[:2]
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
    
    # --- Load keypoints using the function argument ---
    with open(keypoints_path) as f:
        keypoints_json = json.load(f)
    
    # Check if keypoints were found
    if not keypoints_json['people']:
        print(f"Error: No people found in keypoints file {keypoints_path}")
        return

    keypoints = np.array(keypoints_json['people'][0]['pose_keypoints_2d']).reshape(-1, 3)
    
    # --- Define pose pairs ---
    POSE_PAIRS = [
        (1, 2), (1, 5), (2, 3), (3, 4), (5, 6),
        (6, 7), (1, 8), (8, 9), (9, 10), (1, 11),
        (11, 12), (12, 13), (0, 1), (0, 14),
        (14, 16), (0, 15), (15, 17)
    ]
    
    # --- Color gradient for limbs (from red to violet) ---
    def get_color(index, total):
        hsv = np.array([[[int(index / total * 180), 255, 255]]], dtype=np.uint8)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)[0][0]
        return tuple(int(c) for c in bgr)
    
    # --- Draw keypoints, labels, and colored limbs ---
    for i, (x, y, conf) in enumerate(keypoints):
        if conf > 0.1:
            cv2.circle(canvas, (int(x), int(y)), 5, (255, 255, 255), -1)  # white joint
            cv2.putText(canvas, str(i), (int(x)+5, int(y)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    for i, (partA, partB) in enumerate(POSE_PAIRS):
        xA, yA, cA = keypoints[partA]
        xB, yB, cB = keypoints[partB]
        if cA > 0.1 and cB > 0.1:
            ptA = (int(xA), int(yA))
            ptB = (int(xB), int(yB))
            color = get_color(i, len(POSE_PAIRS))  # rainbow color
            cv2.line(canvas, ptA, ptB, color, thickness=10)
    
    # --- Save the final skeleton image ---
    os.makedirs(output_dir, exist_ok=True)
    # Create a dynamic output name based on the input image
    base_name = os.path.basename(image_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_skeleton.png"
    output_path = os.path.join(output_dir, output_filename)
    
    cv2.imwrite(output_path, canvas)
    print(f"✅ Skeleton image saved to: {output_path}")

In [ ]:
import os

def pre_processing(cloth_dir, image_dir):
    """
    Runs the full pre-processing pipeline by pairing sorted files from
    the cloth and image directories.
    """
    print("--- Starting Full Pre-processing Pipeline ---")
    
    # Step 1: Run parsing on the entire image directory
    print("\n[1/4] Running human parsing on all images...")
    parsing_map(image_dir) 
    
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    cloth_files = sorted([f for f in os.listdir(cloth_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

    if len(image_files) != len(cloth_files):
        print(f"Warning: Mismatch in file counts. Images: {len(image_files)}, Cloths: {len(cloth_files)}")
        print("Processing only the minimum number of pairs.")
        
    num_pairs = min(len(image_files), len(cloth_files))
    
    # Loop through the number of pairs
    for i in range(num_pairs):
        image_filename = image_files[i]
        cloth_filename = cloth_files[i]
        
        print(f"\n--- Processing Pair {i+1}/{num_pairs}: ---")
        print(f"  Image: {image_filename}")
        print(f"  Cloth: {cloth_filename}")
        
        # Define paths for the current pair
        image_path = os.path.join(image_dir, image_filename)
        cloth_path = os.path.join(cloth_dir, cloth_filename)

        # Step 2: Create cloth mask
        print(f"[2/4] Creating cloth mask for {cloth_filename}...")
        cloth_mask(cloth_path)

        # Step 3: Create keypoints JSON
        print(f"[3/4] Creating keypoints for {image_filename}...")
        keypoints_json_path = keypoints(image_path)
        
        if keypoints_json_path is None:
            print("Skipping skeleton creation due to no keypoints.")
            continue

        # Step 4: Create OpenPose skeleton image
        print(f"[4/4] Creating skeleton image for {image_filename}...")
        open_pose(image_path, keypoints_json_path)


In [ ]:
# Define your input directories
my_cloth_directory = "/kaggle/input/data-of-user/data/cloth"
my_image_directory = "/kaggle/input/dataset-of-person/data"

# Run the entire pipeline
pre_processing(cloth_dir=my_cloth_directory, image_dir=my_image_directory)